# LINK PREDICTION

## NODE2VEC

In [10]:
import pandas as pd
import networkx as nx
import numpy as np
from node2vec import Node2Vec
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder

# Load country data
file_path = "./data/country.csv"
df = pd.read_csv(file_path)

# Extract country names
countries = list(df['value'])

# Create directed graph
G = nx.DiGraph()
for country in countries:
    last_letter = country[-1].lower()
    for other_country in countries:
        if other_country[0].lower() == last_letter and country != other_country:
            G.add_edge(country, other_country)

# Step 1: Generate Node2Vec Embeddings
node2vec = Node2Vec(G, dimensions=64, walk_length=50, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Get Node2Vec embedding for a node
def get_embedding(node):
    return model.wv[node] if node in model.wv else np.zeros(64)

# Step 2: One-Hot Encode First & Last Letters
letters = sorted(set([c[0].lower() for c in countries] + [c[-1].lower() for c in countries]))
letter_encoder = OneHotEncoder(sparse_output=False, categories=[letters, letters])

# Fit OneHotEncoder
letter_encoder.fit([[c[0].lower(), c[-1].lower()] for c in countries])

# Convert each country to (first_letter, last_letter) encoding
def get_letter_feature(country):
    return letter_encoder.transform([[country[0].lower(), country[-1].lower()]])[0]

# Step 3: Create Feature Vectors for Edges
def create_edge_features(edges):
    edge_features = []
    for u, v in edges:
        emb_u, emb_v = get_embedding(u), get_embedding(v)
        letter_feat_u, letter_feat_v = get_letter_feature(u), get_letter_feature(v)
        edge_features.append(np.concatenate([emb_u, emb_v, letter_feat_u, letter_feat_v]))
    return np.array(edge_features)

# Step 4: Mask Some Edges (20% of real edges for testing)
edges = list(G.edges)
random.shuffle(edges)
split = int(len(edges) * 0.8)
train_edges, test_edges = edges[:split], edges[split:]

# Remove test edges from the graph (Masking)
G_train = G.copy()
G_train.remove_edges_from(test_edges)

# Step 5: Sample Negative Edges (Non-Existing Links)
non_edges = list(nx.non_edges(G))
random.shuffle(non_edges)
train_non_edges = non_edges[:split]
test_non_edges = non_edges[split:]

# Ensure we have positive and negative samples
X_train = np.vstack((create_edge_features(train_edges), create_edge_features(train_non_edges)))
y_train = np.hstack((np.ones(len(train_edges)), np.zeros(len(train_non_edges))))  # 1 = real link, 0 = fake link

X_test = np.vstack((create_edge_features(test_edges), create_edge_features(test_non_edges)))
y_test = np.hstack((np.ones(len(test_edges)), np.zeros(len(test_non_edges))))  # 1 = real link, 0 = fake link

# Step 6: Train a Logistic Regression Model for Link Prediction
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Step 7: Evaluate the Model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"🔹 Node2Vec + Letter Features Accuracy: {accuracy:.4f}")
print(f"🔹 Node2Vec + Letter Features ROC-AUC Score: {roc_auc:.4f}")


Generating walks (CPU: 3): 100%|██████████| 50/50 [00:42<00:00,  1.16it/s]


🔹 Node2Vec + Letter Features Accuracy: 0.8291
🔹 Node2Vec + Letter Features ROC-AUC Score: 0.8692


## GRAPH NEURAL NETWORK

In [11]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.utils import from_networkx, negative_sampling
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Load country data
file_path = "./data/country.csv"
df = pd.read_csv(file_path)

# Extract country names
countries = list(df['value'])

# Create directed graph
G = nx.DiGraph()
for country in countries:
    last_letter = country[-1].lower()
    for other_country in countries:
        if other_country[0].lower() == last_letter and country != other_country:
            G.add_edge(country, other_country)

# Convert NetworkX graph to PyTorch Geometric format
G_undirected = G.to_undirected()
data = from_networkx(G_undirected)

# Step 1: One-Hot Encode First & Last Letters
letters = sorted(set([c[0].lower() for c in countries] + [c[-1].lower() for c in countries]))  # Unique letters
letter_encoder = OneHotEncoder(sparse_output=False, categories=[letters, letters])

# Fit OneHotEncoder
letter_encoder.fit([[c[0].lower(), c[-1].lower()] for c in countries])

# Convert each country to (first_letter, last_letter) encoding
def get_letter_feature(country):
    return letter_encoder.transform([[country[0].lower(), country[-1].lower()]])[0]

# Assign node features (first & last letter encoding)
node_features = np.array([get_letter_feature(c) for c in countries])
data.x = torch.tensor(node_features, dtype=torch.float)

print(f"Graph has {data.num_nodes} nodes and {data.num_edges} edges.")


Graph has 249 nodes and 6878 edges.


In [12]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Define Graph Convolutional Network (GCN) Model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [13]:
import random

# Mask out some edges for training
num_edges = data.edge_index.shape[1]
perm = torch.randperm(num_edges)
train_mask = perm[:int(0.8 * num_edges)]
test_mask = perm[int(0.8 * num_edges):]

train_edges = data.edge_index[:, train_mask]
test_edges = data.edge_index[:, test_mask]

# Generate negative (non-existent) edges
neg_edges = negative_sampling(data.edge_index, num_nodes=data.num_nodes, num_neg_samples=num_edges)

train_neg_edges = neg_edges[:, :train_edges.shape[1]]
test_neg_edges = neg_edges[:, test_edges.shape[1]]


In [ ]:
# Initialize Model & Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(in_channels=data.x.shape[1], hidden_channels=32, out_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()

# Move data to GPU if available
data = data.to(device)
train_edges, train_neg_edges = train_edges.to(device), train_neg_edges.to(device)

# Training loop
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    embeddings = model(data.x, data.edge_index)
    
    # Compute positive and negative scores
    pos_scores = (embeddings[train_edges[0]] * embeddings[train_edges[1]]).sum(dim=1)
    neg_scores = (embeddings[train_neg_edges[0]] * embeddings[train_neg_edges[1]]).sum(dim=1)
    
    # Compute loss  
    loss = loss_fn(pos_scores, torch.ones_like(pos_scores)) + loss_fn(neg_scores, torch.zeros_like(neg_scores))
    
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 1.3796
Epoch 20, Loss: 1.1289
Epoch 40, Loss: 0.9574
Epoch 60, Loss: 0.8345
Epoch 80, Loss: 0.8036
Epoch 100, Loss: 0.7882
Epoch 120, Loss: 0.7670
Epoch 140, Loss: 0.7611
Epoch 160, Loss: 0.7547
Epoch 180, Loss: 0.7512


In [19]:
from sklearn.metrics import roc_auc_score, accuracy_score

# Ensure test edges are on the correct device
test_edges, test_neg_edges = test_edges.to(device), test_neg_edges.to(device)

# Ensure correct shape
test_edges = test_edges.view(2, -1)
test_neg_edges = test_neg_edges.view(2, -1)

# Model evaluation
model.eval()
with torch.no_grad():
    embeddings = model(data.x, data.edge_index)

    # Compute positive and negative scores
    pos_scores = (embeddings[test_edges[0]] * embeddings[test_edges[1]]).sum(dim=1)
    neg_scores = (embeddings[test_neg_edges[0]] * embeddings[test_neg_edges[1]]).sum(dim=1)

    # Ensure score dimensions are valid
    assert pos_scores.dim() == 1, f"Expected pos_scores to be 1D, but got {pos_scores.shape}"
    assert neg_scores.dim() == 1, f"Expected neg_scores to be 1D, but got {neg_scores.shape}"

# Convert scores to probability predictions
y_pred = torch.cat([pos_scores, neg_scores]).sigmoid().cpu().numpy()
y_true = np.hstack([np.ones(len(pos_scores)), np.zeros(len(neg_scores))])

# Compute ROC-AUC Score
roc_auc = roc_auc_score(y_true, y_pred)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred_binary = (y_pred >= 0.5).astype(int)

# Compute Accuracy Score
accuracy = accuracy_score(y_true, y_pred_binary)

print(f"🔹 GNN Link Prediction ROC-AUC Score: {roc_auc:.4f}")
print(f"🔹 GNN Link Prediction Accuracy: {accuracy:.4f}")


🔹 GNN Link Prediction ROC-AUC Score: 0.7682
🔹 GNN Link Prediction Accuracy: 0.9855


## Performances
- Clearly, our GNN far outshines our Node2Vec implementation in terms of accuracy (**98.55%** vs **82.91%**)
- However, when it comes to ROC-AUC score, Node2Vec performed better, giving **0.8692** vs the GNN's **0.7682**

## Analysis & Intuition Behind Feature Choices
- I constructed two different types of features for the models:

    - 1️⃣ Node2Vec Features
        -  Encodes structural similarity by learning dense vector embeddings.
        -  Captures graph topology—countries frequently used in play will have closer embeddings.
        -  Strength: Pretrained embeddings allow fast inference.
        -  Limitation: Doesn't adapt dynamically to changes in the graph.

    - 2️⃣ One-Hot Encoded Letter Features
        -  Uses only the first and last letter of each country as features.
        -  Simple but directly aligns with the Atlas game rule.
        -  Strength: Rule-based and interpretable, less complex than deep embeddings.
        -  Limitation: Ignores graph connectivity beyond first/last letter.

    - 3️⃣ GNN Features
        -  Learns latent structural relationships dynamically through message passing.
        -  Uses both letter features & graph topology to predict links.
        -  Strength: Adaptively learns new relationships.
        -  Limitation: Computationally expensive, requires retraining when the graph updates.

## Unsupervised Training Objective
- Since our dataset lacks explicit labels, I trained both models in an unsupervised manner:

    - Masked Edge Training

        - We removed 20% of edges for validation.
        - Model learned to reconstruct missing links.
        - Tested whether the model correctly predicts masked edges.

    - Negative Sampling (Node2Vec)

        - Since not every pair of countries should be connected, we sampled negative edges.
        - Trained classifier to distinguish real vs. non-existent edges.

    - Contrastive Learning in GNNs

        - Model learns to embed similar nodes closer together in representation space.
        - Link prediction is performed by computing similarity between node embeddings.